In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/banglat5-output/checkpoint-9716/config.json
/kaggle/input/banglat5-output/checkpoint-9716/trainer_state.json
/kaggle/input/banglat5-output/checkpoint-9716/spiece.model
/kaggle/input/banglat5-output/checkpoint-9716/training_args.bin
/kaggle/input/banglat5-output/checkpoint-9716/tokenizer_config.json
/kaggle/input/banglat5-output/checkpoint-9716/scheduler.pt
/kaggle/input/banglat5-output/checkpoint-9716/model.safetensors
/kaggle/input/banglat5-output/checkpoint-9716/special_tokens_map.json
/kaggle/input/banglat5-output/checkpoint-9716/optimizer.pt
/kaggle/input/banglat5-output/checkpoint-9716/rng_state.pth
/kaggle/input/banglat5-output/checkpoint-9716/added_tokens.json
/kaggle/input/banglat5-output/checkpoint-9716/generation_config.json
/kaggle/input/pentabd-transliterated-dataset/val.csv
/kaggle/input/pentabd-transliterated-dataset/train.csv
/kaggle/input/pentabd-transliterated-dataset/test.csv


In [2]:
!pip install wandb -qqq
import wandb

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [4]:
! wandb login $secret_value_0

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
!pip install transformers

In [6]:
from transformers import AutoTokenizer
from transformers import AutoTokenizer, MT5Model, TrainingArguments, Trainer, MT5ForConditionalGeneration, AutoModelForSeq2SeqLM
import torch

# tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model_name = "/kaggle/input/banglat5-output/checkpoint-9716" #"google/mt5-small"  # "csebuetnlp/banglat5_nmt_en_bn"Adjust if using a pre-trained model

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model = MT5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch_device)

2024-06-08 09:05:14.918033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 09:05:14.918157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 09:05:15.044919: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
! pip install git+https://github.com/csebuetnlp/normalizer
from normalizer import normalize

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-b48f0umu
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-b48f0umu
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=fe34424ce24ed123d17c747a824ac9a080ff538503064e31db0dcd6df8e446b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-wouj6l8e/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186460 sha256=fadce948e3874962db986ca1628a960846922cbe

In [8]:
df_train = pd.read_csv("/kaggle/input/pentabd-transliterated-dataset/train.csv")
df_test = pd.read_csv("/kaggle/input/pentabd-transliterated-dataset/test.csv")
df_val =  pd.read_csv("/kaggle/input/pentabd-transliterated-dataset/val.csv")

In [9]:
df_train

,id,dataset,text_transliterated,text_bengali
0,e3839126-f351-4ff2-81b2-0d347a93b2db,penta_trickbd_external,"Phone resulation 480p,vedio recording 1080p! How?","ফোন রেজুলেশন ৪৮০পি, ভিডিও রেকর্ডিং ১০৮০পি! হাও?"
1,b14214ab-a53d-47d4-9e58-298e1b3e6f42,penta_trickbd_external,Flash Tool Ki Letest Verson,ফ্ল্যাশ টুল কি লেটেস্ট ভার্সন
2,20d6a478-daac-4459-b362-cfd076a5ee50,penta_trickbd_external,yes..Oem to Unlock kortey hobe ar root korar p...,ইয়েস..ওএম তো আনলক করতে হবে আর রুট করার পর ও সে...
3,13f5078f-3f6c-4a81-be24-b3ef1cc8c984,penta_trickbd_external,lav nai oi eki joma dite jawai lagbe tokhn tak...,লাভ নাই ওই একি জমা দিতে যাওয়াই লাগবে তখন টাকা ...
4,85b9e667-07a2-4861-bbfa-49b5b3107a79,penta_trickbd_external,Deliverer option nai amar.. ki korbo,ডেলিভারার অপশন নেই আমার.. কি করব
...,...,...,...,...
38859,bc6da3ec-7dc7-4b00-a31f-45e3e3082499,springer_autobacktransliteration,Eder ke mati chapa deya dorkar,এদের কে মাটি চাপা দেয়া দরকার
38860,4c8ce30f-9119-4ebd-9ac0-7049dcb8a17d,springer_autobacktransliteration,Oder ekdin bichar hobe vai,ওদের একদিন বিচার হবে ভাই
38861,c698d5b7-d06f-401b-9eac-c38081f49bb0,springer_autobacktransliteration,Opekkha koren,অপেক্ষা করেন
38862,67d48ea7-85a6-450c-8367-c3c1aa4ecd45,springer_autobacktransliteration,Shomoy ashbe,সময় আসবে


In [10]:
# df_train = df_train[:300]
# df_test = df_test[:40]
# df_val = df_val[:40]


In [11]:
print(df_train.isna().sum())
print(df_test.isna().sum())
print(df_val.isna().sum())

id                     0
dataset                0
text_transliterated    0
text_bengali           0
dtype: int64
id                     0
dataset                0
text_transliterated    0
text_bengali           0
dtype: int64
id                     0
dataset                0
text_transliterated    0
text_bengali           0
dtype: int64


In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources (may need internet connection)
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text, language='english'):
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])

    # Convert to lowercase
    text = text.lower()

    # Remove stopwords (optional, adjust stopword list based on language)
    stop_words = stopwords.words(language)
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

# Clean Banglish and Bengali text
# df['Banglish_Clean'] = df['Banglish'].apply(clean_text)
# df['Bengali_Clean'] = df['Bengali'].apply(clean_text, language='bengali')  # Specify Bengali for stopword removal

# Normalization for Bengali text (replace with your desired normalization function)
def normalize_bengali(text):
    normalized_text = normalize(text)
    return normalized_text

df_train['normalized_bengali'] = df_train['text_bengali'].apply(normalize_bengali)
df_test['normalized_bengali'] = df_test['text_bengali'].apply(normalize_bengali)
df_val['normalized_bengali'] = df_val['text_bengali'].apply(normalize_bengali)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df_train["normalized_bengali"][0]

'ফোন রেজুলেশন ৪৮০পি, ভিডিও রেকর্ডিং ১০৮০পি! হাও?'

In [14]:
def find_max_length(df, column_name):
    # Find the index of the text with the maximum length
    max_length_index = df[column_name].str.len().idxmax()

    # Get the text with the maximum length
    max_length_text = df.loc[max_length_index, column_name]

    # Print the maximum length and the corresponding text
#     print(f"Index of the text with maximum length: {max_length_index}")
#     print(f"Maximum length: {len(max_length_text)}")
#     print(f"Text with maximum length:\n{max_length_text}")
    return len(max_length_text)

# find_max_length(df_train, 'text_bengali')
# print(df_train['text_bengali'][10454])

In [15]:
from datasets import Dataset
def pad_truncate(df):
    max_length = 200
#     print(max_length)
    bengali_tokenized = tokenizer(df['normalized_bengali'].tolist(), padding="max_length", truncation=True)
#     print(bengali_tokenized)
#     max_length = find_max_length(df, 'text_transliterated')
    banglish_tokenized = tokenizer(df['text_transliterated'].tolist(), padding="max_length", truncation=True)

    dataset = Dataset.from_dict({
        "input_ids": banglish_tokenized["input_ids"],
        "attention_mask": banglish_tokenized["attention_mask"],
        "labels": bengali_tokenized["input_ids"]  # Labels are target language tokens
    })
    
    return dataset

train_dataset = pad_truncate(df_train)
# print(banglish, bengali)
# pad_truncate(df_test)
# pad_truncate(df_val)

In [16]:
test_dataset = pad_truncate(df_test)
val_dataset = pad_truncate(df_val)

In [17]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# training_args = TrainingArguments(
#     output_dir="./mt5_banglish_bengali",  # Output directory for checkpoints
#     evaluation_strategy="steps",
#     overwrite_output_dir=True,  # Overwrite existing directory if it exists
#     num_train_epochs=3,  # Adjust based on dataset size and desired accuracy
#     per_device_train_batch_size=2,  # Adjust batch size based on GPU memory
#     save_steps=50,  # Save model checkpoints every 10,000 steps
#     save_total_limit=2,  # Keep only the most recent 2 checkpoints
#     logging_steps=50,  # Log training progress every 500 steps
#     fp16 = True,
#     gradient_accumulation_steps = 6,
#     load_best_model_at_end=True  # Load the best model based on validation metrics
# )

batch_size = 4
args = Seq2SeqTrainingArguments(output_dir="weights",
                        evaluation_strategy="epoch",
                        save_strategy = "epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=5e-5,
                        num_train_epochs=7,
                        weight_decay=0.01,
                        save_total_limit=3,
                        predict_with_generate=True,
                        fp16 = False,
                        gradient_accumulation_steps = 6,
                        save_steps = 50,
                        logging_steps = 50,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
!pip install bert-score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00


In [19]:
from datasets import load_metric

# Load the BERTScore metric
bert_metric = load_metric('bertscore')

/tmp/ipykernel_34/1619319013.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bert_metric = load_metric('bertscore')
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [20]:
def compute_metrics(preds_and_labels):
    preds, labels = preds_and_labels

    # Decode the predictions and labels using the tokenizer, skipping special tokens
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 (masked tokens) in labels with the pad token ID
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Decode the labels using the tokenizer, skipping special tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute BERTScore using decoded predictions and labels
    result = bert_metric.compute(predictions=decoded_preds, references=decoded_labels, lang='bn')
    
    # Return the BERTScore as a dictionary
    return {
      'BERT F1': np.mean(result['f1']),
      'BERT Precision': np.mean(result['precision']),
      'BERT Recall': np.mean(result['recall'])
  }

In [21]:
from transformers import DataCollatorForSeq2Seq

# Instantiate a Seq2Seq model from the specified checkpoint

# Define a data collator for Seq2Seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: fabihahaider4 (penta_global). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [17]:
def predict_output(input_sentence):
    input_ids = tokenizer((input_sentence), return_tensors="pt").input_ids.to("cuda")
    generated_tokens = model.generate(input_ids)
    decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]
    decoded_tokens = normalize(decoded_tokens)

    return decoded_tokens
    
df_test['predictions'] = df_test['text_transliterated'].apply(predict_output)
df_test.to_csv("banglaT5_test.csv", index = False)

df_val['predictions'] = df_val['text_transliterated'].apply(predict_output)
df_val.to_csv("banglaT5_val.csv", index = False)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [18]:
df_test

,id,dataset,text_transliterated,text_bengali,normalized_bengali,predictions
0,f5afeacc-e452-497d-80d4-daf5c3238368,penta_trickbd_external,oi mia watermark a click korle subscribe hobe ...,ঐ মিয়া ওয়াটারমার্ক এ ক্লিক করলে সাবস্ক্রাইব হব...,ঐ মিয়া ওয়াটারমার্ক এ ক্লিক করলে সাবস্ক্রাইব ...,<pad> ওই মিয়া ওয়াটারমার্ক এ ক্লিক করলে সাবস্...
1,14a899f1-e09d-4fc2-ad2d-f92075d5992a,penta_trickbd_external,Eta niyeo post korte hoy??,ফ্লাট নিয়েও পোস্ট করতে হয়??,ফ্লাট নিয়েও পোস্ট করতে হয়??,<pad> এটা নিয়েও পোস্ট করতে হয়??</s>
2,379c0d55-c07f-4832-8f88-686640233377,penta_trickbd_external,flat style bar indicator.apk diye,ফ্লাট স্টাইল indicator.apk দিয়ে,ফ্লাট স্টাইল indicator.apk দিয়ে,<pad> ফ্ল্যাট স্টাইল বার ইনডিকেটর।এপিকে দিয়ে</s>
3,5988a6d1-0bbc-453e-a93f-9e34294f6c09,penta_trickbd_external,ekhane gia ager adress copy kore abar kaj korr...,এখানে গিয়ে আগের অ্যাডরেস কপি করে আবার কাজ করতে...,এখানে গিয়ে আগের অ্যাডরেস কপি করে আবার কাজ করত...,<pad>এখানে গিয়া আগের এডপ্রেস কপি করে আবার কাজ...
4,9f505ba3-ec5f-442c-8e9e-f98d0062c27d,penta_trickbd_external,background pic select kore save korle protol n...,ব্যাকগ্রাউণ্ড পিক সিলেক্ট করে সেভ করলে প্রটোল ...,ব্যাকগ্রাউণ্ড পিক সিলেক্ট করে সেভ করলে প্রটোল ...,<pad> ব্যাকগ্রাউন্ড পিক সিলেক্ট করে সেভ করলে প...
...,...,...,...,...,...,...
2495,71df756f-279b-4454-b9c0-63a88afd941f,penta_trickbd_external,ipl er jonno youtube e jotesto… but Ufa or lal...,আইপিএল এর জন্য ইউটিউবে যথেষ্ট… কিন্তু ইউএফএ বা...,আইপিএল এর জন্য ইউটিউবে যথেষ্ট... কিন্তু ইউএফএ ...,<pad>আইপ এর জন্য ইউটিউব এ যথেষ্ট... বাট ইউফা অ...
2496,aea2444d-7a1a-41a6-9206-84be015c3ebe,penta_trickbd_external,"tnx…bro,,,Happy new year ta sort kate likeswn ...","ধন্যবাদ ভাই,,,হ্যাপি নিউ ইয়ার টা শর্টকাটে লেখস...","ধন্যবাদ ভাই,,,হ্যাপি নিউ ইয়ার টা শর্টকাটে লেখ...","<pad>থ্যাংকস...ব্রো,,,হ্যাপি নিউ ইয়ার টা সর্ট..."
2497,6abf65f5-d78f-41f4-884b-59086c241e03,penta_trickbd_external,Oo taile thik ache ami onno kichu vabchilam.. 🙂,ও তাইলে ঠিক আছে আমি অন্য কিছু ভাবছিলাম.. 🙂,ও তাইলে ঠিক আছে আমি অন্য কিছু ভাবছিলাম.. 🙂,<pad> ওও তাইলে ঠিক আছে আমি অন্য কিছু ভাবছিলাম....
2498,b484d9ea-c523-4f6e-b328-245d5de7ee52,penta_trickbd_external,vai ata sobai jane. tai jkew show hidden file ...,ভাই এটা সবাই জানে। তাই কেউ show hidden file on...,ভাই এটা সবাই জানে। তাই কেউ show hidden file on...,<pad>ভাই এটা সবাই জানে। তাই কেউ শো হিডেন ফাইল ...


In [19]:
df_val

,id,dataset,text_transliterated,text_bengali,normalized_bengali,predictions
0,4e670101-f164-4220-bb23-7a5f9a30b2c6,penta_trickbd_external,oi mb diye Browse kora zabena?,ঐ মোবাইল দিয়ে ব্রাউজ করা যাবে না?,ঐ মোবাইল দিয়ে ব্রাউজ করা যাবে না?,<pad> ওই এমবি দিয়ে ব্রাউজ করা যাবেনা?</s>
1,0bcd5128-96d1-4d8f-8ceb-ab64c5eb82d4,penta_trickbd_external,Bolod marka post….,বলদ মার্কা পোস্ট….,বলদ মার্কা পোস্ট....,<pad> ভালো মার্কা পোস্ট....</s>
2,9e32b31c-5683-4778-90b3-95785b905d2d,penta_trickbd_external,1$=1000captha puron korte hoibe…tahole hobe 1$,1৳=১০০০ক্যাপচা পূরণ করতে হবে…তাহলে হবে 1৳,1৳=১০০০ক্যাপচা পূরণ করতে হবে...তাহলে হবে 1৳,<pad> ১$=১০০০০টাকা পূরণ করতে হবে...তাহলে হবে ১...
3,bf484d13-0f1a-43ac-9703-25687f3806a2,penta_trickbd_external,ei game khln nai..?,এই গেম খেলিনি..?,এই গেম খেলিনি..?,<pad> এই গেম খেলেন নাই..?</s>
4,95f170d8-24cf-4819-ac7b-cdaf090b57c1,penta_trickbd_external,a ta age theke jani,এটা আগে থেকে জানি,এটা আগে থেকে জানি,<pad> এটা আগে থেকে জানি</s>
...,...,...,...,...,...,...
1495,fae94029-a03c-4970-9e26-1a9205f85609,penta_trickbd_external,Bondho kore dise……network busy dekhay,বন্ধ করে দিছে…… নেটওয়ার্ক বিজি দেখায়,বন্ধ করে দিছে...... নেটওয়ার্ক বিজি দেখায়,<pad> বন্ধ করে দিসে......নেটওয়ার্ক বিজি দেখায...
1496,88378e71-ba7f-4245-8e65-faa6f4df72be,penta_trickbd_external,"Vai, Apni Jodi ttf file gulo picsart-fonts a m...","ভাই, আপনি যদি টিটিএফ ফাইল গুলো পিক্সআর্ট-ফন্টস...","ভাই, আপনি যদি টিটিএফ ফাইল গুলো পিক্সআর্ট-ফন্টস...","<pad>ভাই, আপনি যদি টিটিএফ ফাইল গুলো পিকস্ট-ফন্..."
1497,2ed0a273-2ae5-4e93-a2b6-b48e556a1051,penta_trickbd_external,"vai 10 second a kom holeo 10 taka katbe, ami e...","ভাই ১০ সেকেন্ড এ কম হলেও ১০ টাকা কাটবে, আমি এর...","ভাই ১০ সেকেন্ড এ কম হলেও ১০ টাকা কাটবে, আমি এর...","<pad> ভাই ১০ সেকেন্ড এ কম হলেও ১০ টাকা কাটবে, ..."
1498,aae28ebc-016d-4375-a60b-0486e86521f3,penta_trickbd_external,Kaj korche.Thank you,কাজ করছে। থ্যাংক ইউ,কাজ করছে। থ্যাংক ইউ,<pad> কাজ করছে।থ্যাংক ইউ</s>
